In [ ]:
from huggingface_hub import login
login("your_huggingface_token")

In [ ]:
import pandas as pd
from transformers import pipeline

# Load the dataset
file_path = 'updated_customer_feedback.csv'
data = pd.read_csv(file_path)

# Load Hugging Face pipelines
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
area_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define candidate labels for area classification
area_labels = ["Dining", "Reception", "Facilities", "Cleanliness", "General"]

# Function to classify sentiment using Hugging Face
def classify_sentiment(feedback):
    result = sentiment_analyzer(feedback)[0]
    label = result['label']
    if label == "NEGATIVE":
        return "Negative"
    elif label == "POSITIVE":
        return "Positive"
    else:
        return "Neutral"

# Function to classify area using zero-shot classification
def classify_area_llm(feedback):
    result = area_classifier(feedback, candidate_labels=area_labels)
    return result['labels'][0]  # Return the label with the highest score

# Analyze the dataset
def analyze_feedback(data):
    results = []
    for index, row in data.iterrows():
        feedback = row["Reviews"]
        sentiment = classify_sentiment(feedback)
        area = classify_area_llm(feedback)
        results.append({
            "Feedback": feedback,
            "Sentiment": sentiment,
            "Area": area
        })
    return pd.DataFrame(results)

# Generate statistical view
def generate_statistics(results_df):
    sentiment_stats = results_df['Sentiment'].value_counts()
    area_stats = results_df['Area'].value_counts()
    print("Sentiment Statistics:")
    print(sentiment_stats)
    print("\nArea Statistics:")
    print(area_stats)
    return sentiment_stats, area_stats

# Main execution
if __name__ == "__main__":
    # Analyze the data
    results_df = analyze_feedback(data)

    # Generate statistics
    sentiment_stats, area_stats = generate_statistics(results_df)

    # Save results to CSV
    output_csv_path = 'feedback_analysis_results.csv'
    results_df.to_csv(output_csv_path, index=False)
    print(f"Results saved locally to {output_csv_path}")

    # Save results to JSON
    output_json_path = 'feedback_analysis_results.json'
    results_df.to_json(output_json_path, orient='records', lines=True, force_ascii=False)
    print(f"Results saved locally to {output_json_path}")
